In [64]:
from os.path import join as pjoin
import numpy as np
import pandas as pd
from datetime import datetime 
from pytz import timezone
import pytz
#import pendulum

In [42]:
input_filepath = "../data/raw"
train_df = pd.read_csv(input_filepath + "/train.csv")
building_df = pd.read_csv(input_filepath + "/building_metadata.csv")
train_df = train_df.merge(building_df, on = "building_id", how = "left")

In [43]:
site_info = pd.read_csv('../data/site_info.csv', delimiter = ";")
site_info

,site_id,timezone,country_code,location
0,0,US/Eastern,US,"Orlando, FL"
1,1,Europe/London,UK,"UK, Southhampton"
2,2,US/Mountain,US,"Tempe, AZ"
3,3,US/Eastern,US,"Washington, WA"
4,4,US/Pacific,US,"San Francisco, CA"
5,5,Europe/London,UK,"UK, London"
6,6,US/Eastern,US,Philadelphia
7,7,Canada/Eastern,CA,Montreal/Ottawa
8,8,US/Eastern,US,"Orlando, FL"
9,9,US/Central,US,"Austin, TX"


In [44]:
site_info.timezone = site_info.timezone.map(lambda x: pytz.timezone(x))

In [45]:
site_info.timezone[1]

<DstTzInfo 'Europe/London' LMT-1 day, 23:59:00 STD>

In [46]:
train_df.shape

(20216100, 9)

In [47]:
train_df = train_df.merge(site_info, on = "site_id", how = "left")

In [48]:
train_df.shape

(20216100, 12)

In [7]:
def reduce_mem_usage(df, verbose=True):
    """
    Takes an dataframe as argument and adjusts the datatypes of the respective
    columns to reduce memory allocation
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if (c_min > np.iinfo(np.int8).min and
                        c_max < np.iinfo(np.int8).max):
                    df[col] = df[col].astype(np.int8)
                elif (c_min > np.iinfo(np.int16).min and
                      c_max < np.iinfo(np.int16).max):
                    df[col] = df[col].astype(np.int16)
                elif (c_min > np.iinfo(np.int32).min and
                      c_max < np.iinfo(np.int32).max):
                    df[col] = df[col].astype(np.int32)
                elif (c_min > np.iinfo(np.int64).min and
                      c_max < np.iinfo(np.int64).max):
                    df[col] = df[col].astype(np.int64)
            else:
                if (c_min > np.finfo(np.float16).min and
                        c_max < np.finfo(np.float16).max):
                    df[col] = df[col].astype(np.float16)
                elif (c_min > np.finfo(np.float32).min and
                      c_max < np.finfo(np.float32).max):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    reduced_mem = 100 * (start_mem - end_mem) / start_mem
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'
              .format(end_mem, reduced_mem))
    return df


In [49]:
train_df = reduce_mem_usage(train_df)

Mem. usage decreased to 1233.89 Mb (38.5% reduction)


In [9]:
def adjust_column_types(data_frame):
    """
    Takes a data frame and parses certain columns to the desired type.
    """
    data_frame["timestamp"] = pd.to_datetime(data_frame["timestamp"])
    return data_frame

In [50]:
train_df = adjust_column_types(train_df)

In [11]:
train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,timezone,country_code,location
0,0,0,2016-01-01,0.0,0,Education,7432,2008.0,NaN,US/Eastern,US,"Orlando, FL"
1,1,0,2016-01-01,0.0,0,Education,2720,2004.0,NaN,US/Eastern,US,"Orlando, FL"
2,2,0,2016-01-01,0.0,0,Education,5376,1991.0,NaN,US/Eastern,US,"Orlando, FL"
3,3,0,2016-01-01,0.0,0,Education,23685,2002.0,NaN,US/Eastern,US,"Orlando, FL"
4,4,0,2016-01-01,0.0,0,Education,116607,1975.0,NaN,US/Eastern,US,"Orlando, FL"


In [58]:
ts = train_df['timestamp'][2]

In [68]:
pd.to_datetime(ts)

Timestamp('2016-01-01 00:00:00')

In [72]:
datetime.strptime('2016-01-01 00:00:10', "%Y-%d-%m %H:%M:%S")

datetime.datetime(2016, 1, 1, 0, 0, 10)

In [14]:
tz = train_df['timezone'][1]

In [18]:
tz.localize(ts)

Timestamp('2016-01-01 00:00:00-0500', tz='US/Eastern')

In [40]:
train_df['timestamp_local'] = [tz.localize(ts) for tz, ts in zip(train_df['timezone'], train_df['timestamp'])]

KeyboardInterrupt: 

In [66]:
train_df['timestamp_local'] = train_df.apply(lambda x: x.timezone.localize(x.timestamp), axis = 2)

datetime.datetime(2013, 3, 31, 2, 30)

In [73]:
train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,timezone,country_code,location
0,0,0,2016-01-01,0.0,0,Education,7432,2008.0,NaN,US/Eastern,US,"Orlando, FL"
1,1,0,2016-01-01,0.0,0,Education,2720,2004.0,NaN,US/Eastern,US,"Orlando, FL"
2,2,0,2016-01-01,0.0,0,Education,5376,1991.0,NaN,US/Eastern,US,"Orlando, FL"
3,3,0,2016-01-01,0.0,0,Education,23685,2002.0,NaN,US/Eastern,US,"Orlando, FL"
4,4,0,2016-01-01,0.0,0,Education,116607,1975.0,NaN,US/Eastern,US,"Orlando, FL"
